In [2]:
# Stage 0: Load up modules
# sc is availabile
import os
import sys
import json
import pandas as pd
from collections import defaultdict
import hashlib
import re
import shutil # because dbutils doesn't expose it's methods all the time?
import unicodedata
from operator import add
from functools import partial
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
import pyspark.sql.functions
from pyspark.sql.types import *
from pyspark.mllib.feature import Word2Vec
from pyspark.ml.feature import NGram
from pyspark.ml.feature import StopWordsRemover
from functools import partial
from itertools import chain, izip, tee
import nltk
from nltk.corpus import stopwords

try:
  STOP_WORDS = stopwords.words("english") # assumes stripped punctucation ([its, ain, t]), a set
except LookupError:
  nltk.download("stopwords") 
  STOP_WORDS = stopwords.words("english")

class SkillsRawAccess(object):
    """Simple access API to raw skills data on S3"""
    def __init__(self, bucket='dssg-labor', base='mnt', parent=''):
        self.bucket = bucket
        self.base = base
        self.parent = parent
        self.format = 'com.databricks.spark.csv'
      
    def rootpath(self, *args):
        return os.path.join(self, '/', self.base, self.bucket, self.parent, *args)

    def writepath(self, file_name):
        return self.rootpath(file_name)
      
    def readpath(self, file_name):
        return self.rootpath(file_name)

In [3]:
# snl #####################################################
######################################################
######################################################
######################################################
# ... write out labeled versions of the raw corpora
S = SkillsRawAccess(bucket='skills-public', parent='usajobs')

name = 'raw_corpora_a'
S.bucket = 'skills-public'
S.parent = 'machine_learning' + '/preprocess/'
copora_nlp_a = sqlContext.read\
                         .format(S.format)           \
                         .options(delimiter='\t')    \
                         .options(header='true')     \
                         .load(S.writepath(name + '.tsv'))

# snl now I have a processed corpus?

name = 'skills_master_table'
S.bucket = 'skills-public'
S.parent = 'machine_learning' + '/table/'
unique_skills = pd.read_csv('/dbfs' + S.writepath(name + '.tsv'), sep='\t')\
                  .drop_duplicates('Element ID')
  
# snl now I have sklls table?
  
skill_uuid_ = unique_skills.set_index(nlp.transforms[0])['skill_uuid']\
                          .to_dict()
nlp_a_skill_uuid = sc.broadcast(skill_uuid_)  

In [4]:
#print copora_nlp_a.describe()
#print copora_nlp_a.first().document
df = copora_nlp_a.sample(False, 0.00005, seed=0)

In [5]:

# for future: 
#  - N-Grams never cross periods or commas
#  - break N-grams on stop words?
def ngram_creator(doc, N):
    join_underscore = "_".join
    for start_idx in xrange(len(doc)-N+1):
      yield join_underscore(doc[start_idx : start_idx+N])

def ngram_counter(doc, N):
    return Counter(list(ngram_creator(doc.split(),N)))
  
def sum_histograms(counter_list): 
  cnt = Counter()
  for c in counter_list:
    cnt += c
  return cnt


#check the pmi count formula
import math
def pmi(phrase, ngram_hist, unigram_hist, sep='_'):
  min_unigrams = 3
  #extra count for every p in the denominator
  n = len(phrase.split(sep)) 
  word_count = sum(unigram_hist.values())
  norm = 0
  nwcount = 0
  for w in phrase.split(sep):
    norm += math.log(unigram_hist[w])
    nwcount += unigram_hist[w]
  pmi = math.log(ngram_hist[phrase]) + math.log((n-1)*word_count) - norm
  # there should be at least 3 of each unigram on average
  if nwcount/n < min_unigrams:
    pmi = -100
  return pmi




In [6]:
# optimize (the collect?)
unigram_hist = sum_histograms(df.map(lambda x: ngram_counter(x.document, 1)).collect()) 
bigram_hist = sum_histograms(df.map(lambda x: ngram_counter(x.document, 2)).collect()) 
#trigram_hist = sum_histograms(df.map(lambda x: ngram_counter(x.document, 3)).collect()) 


In [7]:
import copy 
bigram_pmi = copy.deepcopy(bigram_hist)
for phrase in bigram_hist:
  bigram_pmi[phrase] = pmi(phrase, bigram_hist, unigram_hist)

print '-----------------pmi'
for word, count in bigram_pmi.most_common(10):
    print word, count
print '-----------------collocation counts'
for word, count in bigram_hist.most_common(10):
    print word, count


In [8]:
# after choosing a cutoff, construct a list and remove the known skills from the list
# use negative md5 numbers as values in a dict of those words
# run the same labeler

In [9]:
# ... using this labeler function
def labeler(document, skill_uuid):
  def label(document, skill_uuid):  
    join_spaces = " ".join # for runtime efficiency

    N = 5  
    doc = document.split()
    doc_len = len(doc)

    start_idx = 0

    # Yield a generator of document skills/non skills that advances
    # index pointer (`offset`) to end of skill or current non skill
    # while yielding that skill uuid or non skill token (controlled by
    # `found_skill` flag) to the callee so they can do whatever with it
    while start_idx < doc_len:
      found_skill = False
      offset = 1

      lookahead = min( N, doc_len - start_idx )
      for idx in range( lookahead, 0, -1):
        ngram = join_spaces(doc[start_idx : start_idx+idx])
        if ngram in skill_uuid:
          found_skill = True
          offset = idx

          yield skill_uuid[ngram]
          break

      if not found_skill:
        yield doc[start_idx]

      start_idx += offset
      
  return ' '.join(list(label(document, skill_uuid)))

## doc test type debug code/stuff
skill_uuid = {'computer programming':'23423423', 'fruit':'343'}
doc = "computer programming fruit You fruit   should be familar with computer programming fruit a"
print labeler(doc, skill_uuid)

In [10]:
labeled_corpora_nlp_a = copora_nlp_a.map(lambda x: labeler(x.document, nlp_a_skill_uuid.value))


In [11]:
name = 'sam_new_labeled_corpora_a'
S.bucket = 'skills-public'
S.parent = 'machine_learning' + '/preprocess/'
S.writepath(name + '.tsv')

labeled_corpora_nlp_a = copora_nlp_a.map(lambda x: labeler(x.document, nlp_a_skill_uuid.value))
labeled_corpora_nlp_a.map(Row('document'))\
                     .toDF()\
                     .write\
                     .format(S.format)           \
                     .mode('overwrite')          \
                     .options(delimiter='\t')    \
                     .options(header='true')     \
                     .save(S.writepath(name + '.tsv'))
######################################################
######################################################
######################################################
######################################################

In [12]:
# ... write out master vocabulary file
# note: update, want skill ngram, context (3 or 4 sentences)